In [ ]:
!pip install ultralytics mediapipe opencv-python-headless==4.8.0.74

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
Reason for being yanked: deprecated, use 4.8.0.76
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 12.1 MB/s eta 0:00:00
  Attemp

In [ ]:
import cv2
import mediapipe as mp
import torch
from ultralytics import YOLO
import numpy as np

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Upload your best.pt file into Colab (left sidebar > Files > Upload)
model = YOLO("best.pt")  # make sure your best.pt is uploaded

# Mediapipe Pose setup
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

NameError: name 'YOLO' is not defined

In [ ]:
def get_triage_label(landmarks, fire_near=False):
    """Decide victim posture and triage score based on landmarks."""
    if not landmarks:
        return "Unknown"

    # Example logic (very simple, can improve later):
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    nose = landmarks[mp_pose.PoseLandmark.NOSE.value]
    left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

    # If wrist above nose → Waving
    if left_wrist.y < nose.y or right_wrist.y < nose.y:
        return "Waving - RESCUE NOW"

    # If ankle close to nose (body horizontal) → Prone
    if abs(left_ankle.y - nose.y) < 0.2:
        return "Prone - HIGH RISK"

    # Otherwise → Standing / Low Risk
    return "Standing - Low Risk" if not fire_near else "Standing - Medium Risk"

In [ ]:
input_video = "bending.mp4"   # <-- replace with your uploaded video file
cap = cv2.VideoCapture(input_video)

# Video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output.mp4", fourcc, 20.0,
                      (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection
    results = model(frame, verbose=False)

    fire_near = False
    for r in results:
        boxes = r.boxes.xyxy.cpu().numpy()
        labels = r.boxes.cls.cpu().numpy()
        confs = r.boxes.conf.cpu().numpy()

        for box, label, conf in zip(boxes, labels, confs):
            x1, y1, x2, y2 = map(int, box)
            cls = model.names[int(label)]

            if cls.lower() in ["fire", "smoke"]:
                cv2.putText(frame, f"{cls} {conf:.2f}", (x1, y1-5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                cv2.rectangle(frame, (x1,y1), (x2,y2), (0,0,255), 2)
                fire_near = True

            elif cls.lower() in ["person", "victim", "human"]:
                # Crop person
                person_crop = frame[y1:y2, x1:x2]
                if person_crop.size == 0:
                    continue

                # Mediapipe Pose
                rgb_crop = cv2.cvtColor(person_crop, cv2.COLOR_BGR2RGB)
                results_pose = pose.process(rgb_crop)

                triage_label = "Unknown"
                if results_pose.pose_landmarks:
                    triage_label = get_triage_label(results_pose.pose_landmarks.landmark, fire_near)

                # Draw person box + triage label
                cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
                cv2.putText(frame, f"{triage_label}", (x1, y1-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

print("Processing complete! Annotated video saved as output.mp4")

Processing complete! Annotated video saved as output.mp4
